In [1]:
import pandas as pd
from statistics import mean, median
import sys
import pprint
import pickle
import csv

In [2]:
pp = pprint.PrettyPrinter(indent=4)

In [3]:
batch_sizes = [2, 4, 8, 16]
batching_alg = ['BATCH4', 'BATCHBISECT', 'BATCHSTOP4']
projects = []
num = 1
project_ytest_lib = {}

In [4]:
def get_results(results):
    l = []
    
    for r in results:
        if r == 'passed':
            l.append(1)
        else:
            l.append(0)
            
    return l

In [5]:
def str_to_list(s):
    if s == '[]':
        return [0]
    l = s[1:-1].split(', ')
    l = [int(x) for x in l]
    return l

In [6]:
def get_empty_library():
    global projects
    ssr_delay_lib = {}
    
    for p in projects:
        ssr_delay_lib[p] = {'BATCHSTOP4':{4:{}, 8:{}, 16:{}}, 'BATCH4':{4:{}}, 'BATCHBISECT':{1:{}, 2:{}, 4:{}, 8:{}, 16:{}}}
    for p in ssr_delay_lib:
            for alg in ssr_delay_lib[p]:
                for b in ssr_delay_lib[p][alg]:
                    for i in range(2,21):
                        ssr_delay_lib[p][alg][b][i] = {}
    
    return ssr_delay_lib
        

In [7]:
def output_values(Y_data):
    Y_t = []
    for e in Y_data:
        if e == 'passed':
            Y_t.append(1)
        else:
            Y_t.append(0) 
    return Y_t

In [8]:
def get_first_failures(df):
    
    results = df['tr_status'].tolist()
    length = len(results)
    verdict = ['keep']
    prev = results[0]
    
    for i in range(1, length):
        if results[i] == 0:
            if prev == 0:
                verdict.append('discard')
                #print(i+1)
            else:
                verdict.append('keep')
        else:
            verdict.append('keep')
        prev = results[i]
    
    df['verdict'] = verdict
    df = df[ df['verdict'] == 'keep' ]
    df.drop('verdict', inplace=True, axis=1)
    return df

In [9]:
def get_empty_counter():
    d = {}
    for i in range(0,21):
        d[i] = 0
    return d

In [10]:
def get_delay_from_ci(ci, y_test, batch_size):
    
    i = 1        
    delay_list = {'sbs_list':[], 'batch_list':[]}
    delay_counter = get_empty_counter()
    
    missed = []
    batch_list = []
    sbs_list = []
    
    b = batch_size - 1
    
    while i < len(ci):

        if ci[i] == 0:

            while len(missed) > 0:
                ind = missed.pop()
                sbs_list.append(i-ind)
                
            batch_list.append(b)
            b -= 1
            if b == -1:
                b = batch_size - 1
                

        if ci[i] == 1:
            if y_test[i] == 0:
                missed.append(i)
        #print(y_test[i], ci[i], sbs_list, batch_list)
        i += 1
    
#     for el in range(0, 21):
#         delay_counter[el] += sbs_list.count(el)
            
#     for el in range(0, 21):
#         delay_counter[el] += batch_list.count(el)
    
    i = len(ci)
    while len(missed) > 0:
        ind = missed.pop()
        sbs_list.append(i-ind)
        #delay_counter[i - ind] += 1
    
    
    delay_list['sbs_list'].extend(sbs_list)
    delay_list['batch_list'].extend(batch_list)
    
    return delay_list

In [11]:
def get_delay_lib(ssr_results):
    
    ssr_delay_lib = get_empty_library()
    
    for x in range(len(ssr_results)):
        row = ssr_results.iloc[x]
        ci = str_to_list(row['ci'])
        batch_size = row['batch_size']
        project = row['project']

        test_file = pd.read_csv('../../data/exact_data/' + project + '.csv') 
        test_file['tr_status'] = output_values(test_file['tr_status'])

        y_test = test_file['tr_status'].tolist()
        
#         delay_counter = get_delay_from_ci(ci, y_test, batch_size)
#         ssr_delay_lib[row['project']][row['algorithm']][row['batch_size']][row['confidence']]['delays'] = delay_counter

        delay_list = get_delay_from_ci(ci, y_test, batch_size)
        ssr_delay_lib[row['project']][row['algorithm']][row['batch_size']][row['confidence']]['delays'] = delay_list
        
    return ssr_delay_lib
        

In [12]:
def get_median_from_counter(d):
    length = sum(d.values())
    half = length/2
    l = []
    i = 0
    while i < 21:
        l.extend([i for x in range(d[i]) ])
        if len(l) > half:
            
            if length%2 == 0:
                m_1 = length//2
                m_2 = m_1 + 1
                med = (l[m_1] + l[m_2])/2
                return med
            else:
                m = length//2
                med = l[m+1]
                return med
        i += 1
                

In [13]:
def get_lines(ssr_results, ssr_delay_lib):
    lines = []
    
    for x in range(len(ssr_results)):
        row = ssr_results.iloc[x]
        d = ssr_delay_lib[row['project']][row['algorithm']][row['batch_size']][row['confidence']]['delays']
        final_list = d['sbs_list'] + d['batch_list']
        if len(final_list) == 0:
            m1 = 0
            m2 = 0
            m3 = 0
        else:
            m1 = median(final_list)
            
            if len(d['sbs_list']) != 0:
                m2 = median(d['sbs_list'])
            else:
                m2 = 0
                
            if len(d['batch_list']) != 0:
                m3 = median(d['batch_list'])
            else:
                m3 = 0
            
        lines.append([row['project'], row['algorithm'], row['batch_size'], row['confidence'], row['project_reqd_builds'], row['testall_size'], d['sbs_list'], d['batch_list'], m1, m2, m3])
    return lines


In [18]:
def get_best_confidence(data, b):
    
    #typecasting
    data['builds_reqd'] = pd.to_numeric(data['builds_reqd'], downcast="float", errors='coerce')   
    
    #setting parameters
    project_list = set(data['project'].tolist())
    batching_alg = set(data['algorithm'].tolist())

    #result file
    result_file = open('redo_best_confidence.csv', 'a+')
    res_writer = csv.writer(result_file)
    
    
    #get best confidences
    for p in project_list:
        
        p_name = p + '.csv'
        alg_scores = {}
        p_data = data[data['project'] == p]
        
        y_test = pd.read_csv('../../data/exact_data/' + p_name)['tr_status'].tolist()
        y_test = output_values(y_test)
        
        for alg in batching_alg:

            alg_data = p_data[p_data['algorithm'] == alg]

            total = alg_data['testall_size'].tolist()[0]
            reqd_list = alg_data['builds_reqd'].tolist()
            median_delays = alg_data['combined_median_delay'].tolist()
            delay_lists = alg_data['sbs_delay_list'].tolist()

            min_distance = sys.maxsize

            best = [p_name, 'ssr', alg, b, 0, 0, [], 0, 0]
            best_i = 0

            for i in range(len(reqd_list)):
                distance = ((reqd_list[i]**2)+(median_delays[i])**2)**0.5
                if distance < min_distance:
                    best[4] = reqd_list[i]
                    best[5] = median_delays[i]
                    best[6] = delay_lists[i]
                    best[7] = total
                    best[8] = i+2
                    best_i = i
                    min_distance = distance
            
            res_writer.writerow(best)

In [19]:
result_file = open('redo_best_confidence.csv', 'w')
res_headers = ['project', 'method', 'algorithm', 'batch_size', 'builds_reqd', 'median_delay', 'delay_list', 'testall_size', 'confidence']
res_writer = csv.writer(result_file)
res_writer.writerow(res_headers)

97

In [20]:
filename = '../trial_results.csv'
batchsizes = [1, 2, 4, 8, 16]
batch_frames = []

ssr_results = pd.read_csv(filename) 
projects = set(ssr_results['project'].tolist())

In [21]:
for b in batchsizes:
    batch_data = ssr_results[ ssr_results['batch_size'] == b]
    ssr_delay_lib = get_delay_lib(batch_data)
    lines = get_lines(batch_data, ssr_delay_lib)
    num += 1
    df = pd.DataFrame(lines, columns=['project', 'algorithm', 'batch_size', 'confidence', 'builds_reqd', 'testall_size', 'sbs_delay_list', 'batching_delay_list', 'combined_median_delay', 'sbs_median_delay', 'batching_median_delay'])
    get_best_confidence(df, b)
    batch_frames.append(df)
    

/var/folders/09/2vzp5xpd2ldd4ggk4xzjt1qr0000gn/T/ipykernel_48148/2318945686.py:3: DtypeWarning: Columns (4,7,10,14,19,41,59) have mixed types.Specify dtype option on import or set low_memory=False.
  ssr_delay_lib = get_delay_lib(batch_data)
/var/folders/09/2vzp5xpd2ldd4ggk4xzjt1qr0000gn/T/ipykernel_48148/2318945686.py:7: DtypeWarning: Columns (4,7,10,14,19,41,59) have mixed types.Specify dtype option on import or set low_memory=False.
  get_best_confidence(df, b)


In [22]:
new_df = pd.DataFrame()
for b in batch_frames:
    new_df = new_df.append(b)

In [23]:
new_df.to_csv('sbs_delays_ssr.csv')

In [ ]:
# new_df = final_df.drop('sbs_delay_list', axis=1)


# Combining Versions

In [13]:
def separate_versions(results):
    version_dfs = []
    
    for i in range(1,11):
        ver = results[ results['version']==i]
        version_dfs.append(ver)
    return version_dfs

In [14]:
def combine_versions(results):
    
    projects = set(results['project'].tolist())
    all_versions = pd.DataFrame()
        
    for p in projects:
        
        p_data = results[ results['project']==p]
        pframe = pd.DataFrame()
        
        versions = separate_versions(p_data)
        
        for start in range(0,10):
            if len(versions[start]) > 0:
                pframe = versions[start]
                break
        
        for x in range(len(pframe)):
            row = pframe.iloc[x]
            
            alg = row['algorithm']
            b = row['batch_size']
            conf = row['confidence']
            
            final_delay_list = row['delay_list']
            
            for i in range(start+1,10):
                next_ver = versions[i]
                new_df = next_ver[ (next_ver['algorithm']==alg) & (next_ver['batch_size']==b) & (next_ver['confidence']==conf)]
                
                if len(new_df) > 0:
                    new_row = new_df.iloc[0]
                else:
                    continue
                
                total_size = row['testall_size'] + new_row['testall_size']
                total_reqd_builds = (row['builds_reqd']*row['testall_size']) + (new_row['builds_reqd']*new_row['testall_size'])
                final_delay_list.extend(new_row['delay_list'])
                
                row['builds_reqd'] = total_reqd_builds/total_size                
                row['delay_list'] = final_delay_list
                row['testall_size'] = total_size
                
            pframe.iloc[x] = row
        all_versions = all_versions.append(pframe)
    
    return all_versions

In [15]:
delay_df = combine_versions(final_df)


/opt/homebrew/Caskroom/miniforge/base/envs/new_env/lib/python3.9/site-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()
/opt/homebrew/Caskroom/miniforge/base/envs/new_env/lib/python3.9/site-packages/pandas/core/indexing.py:1797: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, v, pi)


In [16]:
delay_df.drop('delay_list', inplace=True, axis=1)

In [17]:
delay_df.to_csv('full_combined_delay.csv')